# **Derive Inclusion Set from procedure_events file**

The procedure_events file records ventilation and intubation/extubation events under ordercategoryname column.

By filtering out for just those patients we can get the inclusion set of patients by applying the following filters:

**Filter 1: Needs to be in the ICU**
- Given we are in the ICU folder this is implied

**Filter 2: Needs to have undergone invasive mechanical ventilation**
- Filter out patients that have a ventilation event recorded under ordercategoryname column with itemid 225792 = Invasive Ventilation

**Filter 3: Needs to have been extubated**
- Where patient has event logged with itemid 227194 = Extubation

This will give a list of patients with one or more stays where they underwent invasive mechanical ventilation and were extubated.

**Section 1 - Read in file**

**1.1 - Read the file into dataframe**

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/mimic-iv-2.2-raw-data/icu/procedureevents.csv'

procedure_events = pd.read_csv(file_path)

In [ ]:
procedure_events.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10000032,29079034,39553978,88981.0,2180-07-23 14:43:00,2180-07-23 14:44:00,2180-07-23 14:43:00,225966,1.0,NaN,...,6416557,6416557,Procedures,Task,39.4,0,0,FinishedRunning,1.0,0
1,10000032,29079034,39553978,NaN,2180-07-23 14:24:00,2180-07-23 23:50:00,2180-07-23 23:50:49.983,224275,566.0,min,...,6497934,6497934,Peripheral Lines,ContinuousProcess,39.4,1,0,FinishedRunning,566.0,1
2,10000032,29079034,39553978,NaN,2180-07-23 14:24:00,2180-07-23 23:50:00,2180-07-23 23:50:49.983,224277,566.0,min,...,9643097,9643097,Peripheral Lines,ContinuousProcess,39.4,1,0,FinishedRunning,566.0,1
3,10000980,26913865,39765666,NaN,2189-06-27 09:01:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,225794,697.0,min,...,5989583,5989583,Ventilation,ContinuousProcess,76.2,1,0,FinishedRunning,697.0,1
4,10000980,26913865,39765666,NaN,2189-06-27 09:15:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,224277,683.0,min,...,476764,476764,Peripheral Lines,ContinuousProcess,76.2,1,0,FinishedRunning,683.0,1


In [ ]:
procedure_events.describe()

,subject_id,hadm_id,stay_id,caregiver_id,itemid,value,orderid,linkorderid,patientweight,isopenbag,continueinnextdept,originalamount,originalrate
count,6.960920e+05,6.960920e+05,6.960920e+05,562488.000000,696092.000000,696092.000000,6.960920e+05,6.960920e+05,696092.000000,696092.000000,696092.000000,696092.000000,696092.000000
mean,1.500076e+07,2.500131e+07,3.499897e+07,51694.393232,225229.190380,1477.055239,4.998536e+06,4.998577e+06,82.978190,0.465776,0.000250,1490.736688,0.486291
std,2.889455e+06,2.874827e+06,2.893103e+06,29684.862418,1720.069998,3355.171428,2.888588e+06,2.888544e+06,70.247777,0.498828,0.015808,3381.059721,0.499812
min,1.000003e+07,2.000009e+07,3.000015e+07,29.000000,221214.000000,0.000694,2.000000e+01,2.000000e+01,0.100000,0.000000,0.000000,0.000694,0.000000
25%,1.249125e+07,2.251556e+07,3.248311e+07,26640.000000,224275.000000,1.000000,2.497603e+06,2.497672e+06,66.700000,0.000000,0.000000,1.000000,0.000000
50%,1.500373e+07,2.500807e+07,3.499835e+07,53217.000000,225402.000000,1.000000,4.994732e+06,4.994881e+06,79.400000,0.000000,0.000000,1.000000,0.000000
75%,1.750549e+07,2.747480e+07,3.750139e+07,80743.000000,225752.000000,1676.000000,7.502978e+06,7.502914e+06,94.800000,1.000000,0.000000,1690.000000,1.000000
max,1.999999e+07,2.999983e+07,3.999981e+07,99944.000000,229755.000000,143384.000000,9.999994e+06,9.999994e+06,14109.000000,1.000000,1.000000,143384.000000,1.000000


In [ ]:
procedure_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696092 entries, 0 to 696091
Data columns (total 22 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   subject_id                696092 non-null  int64  
 1   hadm_id                   696092 non-null  int64  
 2   stay_id                   696092 non-null  int64  
 3   caregiver_id              562488 non-null  float64
 4   starttime                 696092 non-null  object 
 5   endtime                   696092 non-null  object 
 6   storetime                 696092 non-null  object 
 7   itemid                    696092 non-null  int64  
 8   value                     696092 non-null  float64
 9   valueuom                  338500 non-null  object 
 10  location                  152930 non-null  object 
 11  locationcategory          152930 non-null  object 
 12  orderid                   696092 non-null  int64  
 13  linkorderid               696092 non-null  i

In [ ]:
procedure_events.shape

(696092, 22)

**1.2 - Apply Filter 1**

In [ ]:
# Already applied given we are in ICU directory

**1.3 - Apply Filter 2**

In [ ]:
ventilation = procedure_events[procedure_events['ordercategoryname'] == 'Ventilation']

ventilation.shape

(33721, 22)

In [ ]:
ventilation.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
3,10000980,26913865,39765666,NaN,2189-06-27 09:01:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,225794,697.0,min,...,5989583,5989583,Ventilation,ContinuousProcess,76.2,1,0,FinishedRunning,697.0,1
14,10001884,26184834,37510196,31763.0,2131-01-12 21:30:00,2131-01-13 04:00:00,2131-01-15 04:07:00,225794,390.0,min,...,4809276,4809276,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,390.0,1
20,10001884,26184834,37510196,42150.0,2131-01-11 04:40:00,2131-01-12 17:40:00,2131-01-12 17:49:00,225792,2220.0,min,...,3830120,3830120,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,2220.0,1
27,10001884,26184834,37510196,91332.0,2131-01-13 04:00:00,2131-01-19 17:45:00,2131-01-19 18:44:00,225792,9465.0,min,...,4465887,4465887,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,9465.0,1
28,10001884,26184834,37510196,91332.0,2131-01-15 04:07:00,2131-01-19 17:43:00,2131-01-19 18:44:00,225792,6576.0,min,...,1861924,1861924,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,6576.0,1


In [ ]:
# Filter for itemid 225792 = 'Invasive ventilation'
invasive_ventilation = ventilation[ventilation['itemid'] == 225792]

invasive_ventilation.shape

(30710, 22)

In [ ]:
invasive_ventilation.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
20,10001884,26184834,37510196,42150.0,2131-01-11 04:40:00,2131-01-12 17:40:00,2131-01-12 17:49:00,225792,2220.0,min,...,3830120,3830120,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,2220.0,1
27,10001884,26184834,37510196,91332.0,2131-01-13 04:00:00,2131-01-19 17:45:00,2131-01-19 18:44:00,225792,9465.0,min,...,4465887,4465887,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,9465.0,1
28,10001884,26184834,37510196,91332.0,2131-01-15 04:07:00,2131-01-19 17:43:00,2131-01-19 18:44:00,225792,6576.0,min,...,1861924,1861924,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,6576.0,1
33,10002013,23581541,39060235,27479.0,2160-05-18 14:19:00,2160-05-18 18:01:00,2160-05-18 18:39:00,225792,222.0,min,...,4169380,4169380,Ventilation,ContinuousProcess,96.0,1,0,FinishedRunning,222.0,1
80,10002428,23473524,35479615,27479.0,2156-05-11 16:05:00,2156-05-20 10:45:00,2156-05-20 10:51:00,225792,12640.0,min,...,3976442,3976442,Ventilation,ContinuousProcess,48.4,1,0,FinishedRunning,12640.0,1


In [ ]:
invasive_ventilation_stays = invasive_ventilation[['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid', 'ordercategoryname']]

invasive_ventilation_stays.head()

,subject_id,hadm_id,stay_id,starttime,endtime,itemid,ordercategoryname
20,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation
27,10001884,26184834,37510196,2131-01-13 04:00:00,2131-01-19 17:45:00,225792,Ventilation
28,10001884,26184834,37510196,2131-01-15 04:07:00,2131-01-19 17:43:00,225792,Ventilation
33,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,225792,Ventilation
80,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation


30710 invasive ventilation events across patients and stays

**1.4 - Apply Filter 3**

In [ ]:
# Filter by ordercategoryname = 'Ventilation' or 'Intubation/Extubation'
filter_category_df = procedure_events[
    (procedure_events['ordercategoryname'] == 'Ventilation') |
    (procedure_events['ordercategoryname'] == 'Intubation/Extubation')
]

filter_category_df.shape[0]

64545

In [ ]:
filter_category_df.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
3,10000980,26913865,39765666,NaN,2189-06-27 09:01:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,225794,697.0,min,...,5989583,5989583,Ventilation,ContinuousProcess,76.2,1,0,FinishedRunning,697.0,1
14,10001884,26184834,37510196,31763.0,2131-01-12 21:30:00,2131-01-13 04:00:00,2131-01-15 04:07:00,225794,390.0,min,...,4809276,4809276,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,390.0,1
15,10001884,26184834,37510196,35966.0,2131-01-12 17:40:00,2131-01-12 17:41:00,2131-01-12 17:50:00,227194,1.0,NaN,...,6470885,6470885,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0
19,10001884,26184834,37510196,42150.0,2131-01-11 04:30:00,2131-01-11 04:31:00,2131-01-11 05:40:00,224385,1.0,NaN,...,2338046,2338046,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0
20,10001884,26184834,37510196,42150.0,2131-01-11 04:40:00,2131-01-12 17:40:00,2131-01-12 17:49:00,225792,2220.0,min,...,3830120,3830120,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,2220.0,1


In [ ]:
# Filter by itemid 225792 = 'Invasive ventilation' or 227194 = 'Extubation'
filter_itemid_df = filter_category_df[
    (filter_category_df['itemid'] == 225792) |
    (filter_category_df['itemid'] == 227194)
]

filter_itemid_df.shape[0]

53046

In [ ]:
filter_itemid_df.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
15,10001884,26184834,37510196,35966.0,2131-01-12 17:40:00,2131-01-12 17:41:00,2131-01-12 17:50:00,227194,1.0,NaN,...,6470885,6470885,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0
20,10001884,26184834,37510196,42150.0,2131-01-11 04:40:00,2131-01-12 17:40:00,2131-01-12 17:49:00,225792,2220.0,min,...,3830120,3830120,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,2220.0,1
27,10001884,26184834,37510196,91332.0,2131-01-13 04:00:00,2131-01-19 17:45:00,2131-01-19 18:44:00,225792,9465.0,min,...,4465887,4465887,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,9465.0,1
28,10001884,26184834,37510196,91332.0,2131-01-15 04:07:00,2131-01-19 17:43:00,2131-01-19 18:44:00,225792,6576.0,min,...,1861924,1861924,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,6576.0,1
33,10002013,23581541,39060235,27479.0,2160-05-18 14:19:00,2160-05-18 18:01:00,2160-05-18 18:39:00,225792,222.0,min,...,4169380,4169380,Ventilation,ContinuousProcess,96.0,1,0,FinishedRunning,222.0,1


Count the number of extubation events in the set of patients undergoing invasive ventilation

In [ ]:
# Count the number of entries for each item_id
itemid_counts = filter_itemid_df['itemid'].value_counts()

count_extubation = itemid_counts.get(227194, 0)

print(f"Number of entries with itemid 227194: {count_extubation}")

Number of entries with itemid 227194: 22336


**Dataframe refinement to get more focused table**

Only need subject_id, hadm_id, stay_id, starttime, endtime, itemid and ordercategoryname columns.

In [ ]:
imv_extubation_df = filter_itemid_df[['subject_id', 'hadm_id', 'stay_id', 'starttime', 'endtime', 'itemid', 'ordercategoryname']]

imv_extubation_df.head()

,subject_id,hadm_id,stay_id,starttime,endtime,itemid,ordercategoryname
15,10001884,26184834,37510196,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
20,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation
27,10001884,26184834,37510196,2131-01-13 04:00:00,2131-01-19 17:45:00,225792,Ventilation
28,10001884,26184834,37510196,2131-01-15 04:07:00,2131-01-19 17:43:00,225792,Ventilation
33,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,225792,Ventilation


In [ ]:
imv_extubation_df.shape[0]

53046

Save initial dataframe as parquet file to save current state

In [ ]:
!pip install pyarrow

In [ ]:
# parquet_file_path = '/content/drive/MyDrive/MSc_Final_Project/mimic_data_extraction/mimic_extracted_data/vent_and_extub_full_inclusion_working_set_v01.parquet'

# imv_extubation_df.to_parquet(parquet_file_path, index=False)

Having filtered out the initial working set we can now determine the final working set which comprises only ventilation events that have a corresponding extubation event.

Ventilation events that did not result in an extubation attempt will not be considered in this study.



The logic is as follows:
- If the starttime of extubation is within 10 mins after the endtime of ventilation where the subject_id and stay_id are the same, this counts as ventilation with attempted extubation

In [ ]:
# Convert starttime and endtime to datetime objects
imv_extubation_df['starttime'] = pd.to_datetime(imv_extubation_df['starttime'])
imv_extubation_df['endtime'] = pd.to_datetime(imv_extubation_df['endtime'])

imv_extubation_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53046 entries, 15 to 696079
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   subject_id         53046 non-null  int64         
 1   hadm_id            53046 non-null  int64         
 2   stay_id            53046 non-null  int64         
 3   starttime          53046 non-null  datetime64[ns]
 4   endtime            53046 non-null  datetime64[ns]
 5   itemid             53046 non-null  int64         
 6   ordercategoryname  53046 non-null  object        
dtypes: datetime64[ns](2), int64(4), object(1)
memory usage: 5.3+ MB


<ipython-input-27-8cf48fa3faca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imv_extubation_df['starttime'] = pd.to_datetime(imv_extubation_df['starttime'])
<ipython-input-27-8cf48fa3faca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imv_extubation_df['endtime'] = pd.to_datetime(imv_extubation_df['endtime'])


In [ ]:
# Filter for ventilation events
ventilation_df = imv_extubation_df[imv_extubation_df['ordercategoryname'] == 'Ventilation']

# Filter for extubation events
extubation_df = imv_extubation_df[imv_extubation_df['ordercategoryname'] == 'Intubation/Extubation']

In [ ]:
ventilation_df.head()

,subject_id,hadm_id,stay_id,starttime,endtime,itemid,ordercategoryname
20,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation
27,10001884,26184834,37510196,2131-01-13 04:00:00,2131-01-19 17:45:00,225792,Ventilation
28,10001884,26184834,37510196,2131-01-15 04:07:00,2131-01-19 17:43:00,225792,Ventilation
33,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,225792,Ventilation
80,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation


In [ ]:
extubation_df.head()

,subject_id,hadm_id,stay_id,starttime,endtime,itemid,ordercategoryname
15,10001884,26184834,37510196,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
40,10002013,23581541,39060235,2160-05-18 18:00:00,2160-05-18 18:01:00,227194,Intubation/Extubation
83,10002428,23473524,35479615,2156-05-20 10:45:00,2156-05-20 10:46:00,227194,Intubation/Extubation
94,10002428,28662225,38875437,2156-04-22 17:10:00,2156-04-22 17:11:00,227194,Intubation/Extubation
140,10002760,28094813,31831386,2141-04-21 08:00:00,2141-04-21 08:01:00,227194,Intubation/Extubation


In [ ]:
# Merge/Join on 'subject_id' and 'stay_id'
merged_df = pd.merge(
    ventilation_df,
    extubation_df,
    on=['subject_id', 'stay_id'],
    suffixes=('_vent', '_ext')
)

In [ ]:
merged_df.head()

,subject_id,hadm_id_vent,stay_id,starttime_vent,endtime_vent,itemid_vent,ordercategoryname_vent,hadm_id_ext,starttime_ext,endtime_ext,itemid_ext,ordercategoryname_ext
0,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
1,10001884,26184834,37510196,2131-01-13 04:00:00,2131-01-19 17:45:00,225792,Ventilation,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
2,10001884,26184834,37510196,2131-01-15 04:07:00,2131-01-19 17:43:00,225792,Ventilation,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
3,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,225792,Ventilation,23581541,2160-05-18 18:00:00,2160-05-18 18:01:00,227194,Intubation/Extubation
4,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation,23473524,2156-05-20 10:45:00,2156-05-20 10:46:00,227194,Intubation/Extubation


In [ ]:
# Filter out pairs with valid timing between the end of ventilation and the start of extubation in minutes
merged_df['time_diff_vent_ext'] = (merged_df['starttime_ext'] - merged_df['endtime_vent']).dt.total_seconds() / 60

ventilation_with_extubation_df = merged_df[merged_df['time_diff_vent_ext'] <= 10]

In [ ]:
ventilation_with_extubation_df.shape[0]

22548

In [ ]:
ventilation_with_extubation_df.head()

,subject_id,hadm_id_vent,stay_id,starttime_vent,endtime_vent,itemid_vent,ordercategoryname_vent,hadm_id_ext,starttime_ext,endtime_ext,itemid_ext,ordercategoryname_ext,time_diff_vent_ext
0,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation,0.0
1,10001884,26184834,37510196,2131-01-13 04:00:00,2131-01-19 17:45:00,225792,Ventilation,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation,-10085.0
2,10001884,26184834,37510196,2131-01-15 04:07:00,2131-01-19 17:43:00,225792,Ventilation,26184834,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation,-10083.0
3,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,225792,Ventilation,23581541,2160-05-18 18:00:00,2160-05-18 18:01:00,227194,Intubation/Extubation,-1.0
4,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation,23473524,2156-05-20 10:45:00,2156-05-20 10:46:00,227194,Intubation/Extubation,0.0


In [ ]:
# Rename columns for final data frame
ventilation_with_extubation_df = ventilation_with_extubation_df[['subject_id', 'hadm_id_vent', 'stay_id', 'starttime_vent', 'endtime_vent', 'itemid_vent', 'ordercategoryname_vent',
                           'starttime_ext', 'endtime_ext', 'itemid_ext', 'ordercategoryname_ext']]

ventilation_with_extubation_df = ventilation_with_extubation_df.rename(columns={
    'hadm_id_vent': 'hadm_id',
    'starttime_vent': 'ventilation_starttime',
    'endtime_vent': 'ventilation_endtime',
    'itemid_vent': 'ventilation_itemid',
    'ordercategoryname_vent': 'ventilation_ordercategoryname',
    'starttime_ext': 'extubation_starttime',
    'endtime_ext': 'extubation_endtime',
    'itemid_ext': 'extubation_itemid',
    'ordercategoryname_ext': 'extubation_ordercategoryname'
})

In [ ]:
ventilation_with_extubation_df.head()

,subject_id,hadm_id,stay_id,ventilation_starttime,ventilation_endtime,ventilation_itemid,ventilation_ordercategoryname,extubation_starttime,extubation_endtime,extubation_itemid,extubation_ordercategoryname
0,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
1,10001884,26184834,37510196,2131-01-13 04:00:00,2131-01-19 17:45:00,225792,Ventilation,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
2,10001884,26184834,37510196,2131-01-15 04:07:00,2131-01-19 17:43:00,225792,Ventilation,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
3,10002013,23581541,39060235,2160-05-18 14:19:00,2160-05-18 18:01:00,225792,Ventilation,2160-05-18 18:00:00,2160-05-18 18:01:00,227194,Intubation/Extubation
4,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation,2156-05-20 10:45:00,2156-05-20 10:46:00,227194,Intubation/Extubation


In [ ]:
# # Save as CSV file for visualisation
# csv_file_path = '/content/drive/MyDrive/MSc_Final_Project/mimic_data_extraction/mimic_extracted_data/vent_and_extub_merged_df_v01.csv'

# ventilation_with_extubation_df.to_csv(csv_file_path, index=False)

Currently have 22548 ventilation events with extubation but in the original dataset there are only 22336 extubation events in total.

Logically, it is not possible to have more ventilation events with extubation than extubation events.

Hence, we will now attempt to merge again and remove duplicates to see if the numbers match up

In [ ]:
imv_extubation_df['starttime'] = pd.to_datetime(imv_extubation_df['starttime'])
imv_extubation_df['endtime'] = pd.to_datetime(imv_extubation_df['endtime'])

<ipython-input-39-8920c087a044>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imv_extubation_df['starttime'] = pd.to_datetime(imv_extubation_df['starttime'])
<ipython-input-39-8920c087a044>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imv_extubation_df['endtime'] = pd.to_datetime(imv_extubation_df['endtime'])


In [ ]:
# Filter for ventilation events
ventilation_df = imv_extubation_df[imv_extubation_df['ordercategoryname'] == 'Ventilation']

# Filter for extubation events
extubation_df = imv_extubation_df[imv_extubation_df['ordercategoryname'] == 'Intubation/Extubation']

In [ ]:
from datetime import timedelta

In [ ]:
# Initialise list to store valid pairs
valid_pairs = []

# Iterate through each extubation event
for _, ext_row in extubation_df.iterrows():
  subject_id = ext_row['subject_id']
  stay_id = ext_row['stay_id']
  ext_starttime = ext_row['starttime']

  # Filter ventilation events for the same subject_id and stay_id
  subject_ventilation_df = ventilation_df[(ventilation_df['subject_id'] == subject_id) & (ventilation_df['stay_id'] == stay_id)]

  # Find ventilation events that end before extubation starts and within the 10 minute window
  valid_vents = subject_ventilation_df[(subject_ventilation_df['endtime'] <= ext_starttime) &
                                         (subject_ventilation_df['endtime'] >= (ext_starttime - timedelta(minutes=10)))]

  if not valid_vents.empty:
    # Get the most recent ventilation event (closest to the extubation start time)
    nearest_vent = valid_vents.loc[valid_vents['endtime'].idxmax()]

    # Add this pair to the list
    valid_pairs.append({
            'subject_id': subject_id,
            'hadm_id': nearest_vent['hadm_id'],
            'stay_id': stay_id,
            'ventilation_starttime': nearest_vent['starttime'],
            'ventilation_endtime': nearest_vent['endtime'],
            'ventilation_itemid': nearest_vent['itemid'],
            'ventilation_ordercategoryname': nearest_vent['ordercategoryname'],
            'extubation_starttime': ext_starttime,
            'extubation_endtime': ext_row['endtime'],
            'extubation_itemid': ext_row['itemid'],
            'extubation_ordercategoryname': ext_row['ordercategoryname']
        })

unique_vent_with_extub_df = pd.DataFrame(valid_pairs)

In [ ]:
unique_vent_with_extub_df.head()

,subject_id,hadm_id,stay_id,ventilation_starttime,ventilation_endtime,ventilation_itemid,ventilation_ordercategoryname,extubation_starttime,extubation_endtime,extubation_itemid,extubation_ordercategoryname
0,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
1,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation,2156-05-20 10:45:00,2156-05-20 10:46:00,227194,Intubation/Extubation
2,10002428,28662225,38875437,2156-04-19 20:10:00,2156-04-22 17:05:00,225792,Ventilation,2156-04-22 17:10:00,2156-04-22 17:11:00,227194,Intubation/Extubation
3,10002760,28094813,31831386,2141-04-20 14:36:00,2141-04-21 08:00:00,225792,Ventilation,2141-04-21 08:00:00,2141-04-21 08:01:00,227194,Intubation/Extubation
4,10004235,24181354,34100191,2196-02-24 16:52:00,2196-02-27 16:28:00,225792,Ventilation,2196-02-27 16:28:00,2196-02-27 16:29:00,227194,Intubation/Extubation


In [ ]:
unique_events = unique_vent_with_extub_df.shape[0]

print(f"There are {unique_events} invasive ventilation events with extubation")

There are 16300 invasive ventilation events with extubation


In [ ]:
# # Save as CSV file for visualisation
# csv_file_path = '/content/drive/MyDrive/MSc_Final_Project/mimic_data_extraction/mimic_extracted_data/vent_and_extub_merged_duplicates_removed_df_v01.csv'

# unique_vent_with_extub_df.to_csv(csv_file_path, index=False)

Confirmimg this dataset has no duplicate extubation events

In [ ]:
duplicate_extubations = unique_vent_with_extub_df.duplicated(subset=['subject_id', 'stay_id', 'extubation_starttime'], keep=False)

duplicates_df = unique_vent_with_extub_df[duplicate_extubations]

# Check if there are any duplicates
if duplicates_df.empty:
  print("No duplicate extubation events found.")
else:
  print(f"Found {len(duplicates_df)} duplicate extubation events:")
  print(duplicates_df.head())


Found 113 duplicate extubation events:
     subject_id   hadm_id   stay_id ventilation_starttime ventilation_endtime  \
253    10173670  21801929  32131292   2125-02-02 13:30:00 2125-02-08 12:30:00   
254    10173670  21801929  32131292   2125-02-02 13:30:00 2125-02-08 12:30:00   
557    10332371  29819591  30813888   2149-03-06 11:05:00 2149-03-21 12:02:00   
558    10332371  29819591  30813888   2149-03-06 11:05:00 2149-03-21 12:02:00   
691    10418733  20216471  31026358   2119-11-28 13:30:00 2119-12-01 10:15:00   

     ventilation_itemid ventilation_ordercategoryname extubation_starttime  \
253              225792                   Ventilation  2125-02-08 12:30:00   
254              225792                   Ventilation  2125-02-08 12:30:00   
557              225792                   Ventilation  2149-03-21 12:04:00   
558              225792                   Ventilation  2149-03-21 12:04:00   
691              225792                   Ventilation  2119-12-01 10:15:00   

     

There are 113 duplicate extubation events in this dataframe. Thus, these need to be removed.

In [ ]:
# Drop duplicate extubation events, keeping the first occurence
final_df_unique_vent_extub = unique_vent_with_extub_df.drop_duplicates(subset=['subject_id', 'stay_id', 'extubation_starttime'], keep='first')

# Verify that there are no duplicates
duplicate_extubations_after = final_df_unique_vent_extub.duplicated(subset=['subject_id', 'stay_id', 'extubation_starttime'], keep=False)

# Check there are no duplicates left
if duplicate_extubations_after.any():
  print("There are still some duplicate extubation events.")
else:
  print("All duplicate extubation events have been removed.")

All duplicate extubation events have been removed.


In [ ]:
final_df_unique_vent_extub.head()

,subject_id,hadm_id,stay_id,ventilation_starttime,ventilation_endtime,ventilation_itemid,ventilation_ordercategoryname,extubation_starttime,extubation_endtime,extubation_itemid,extubation_ordercategoryname
0,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
1,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation,2156-05-20 10:45:00,2156-05-20 10:46:00,227194,Intubation/Extubation
2,10002428,28662225,38875437,2156-04-19 20:10:00,2156-04-22 17:05:00,225792,Ventilation,2156-04-22 17:10:00,2156-04-22 17:11:00,227194,Intubation/Extubation
3,10002760,28094813,31831386,2141-04-20 14:36:00,2141-04-21 08:00:00,225792,Ventilation,2141-04-21 08:00:00,2141-04-21 08:01:00,227194,Intubation/Extubation
4,10004235,24181354,34100191,2196-02-24 16:52:00,2196-02-27 16:28:00,225792,Ventilation,2196-02-27 16:28:00,2196-02-27 16:29:00,227194,Intubation/Extubation


In [ ]:
unique_extub_events = final_df_unique_vent_extub.shape[0]

print(f"There are {unique_extub_events} invasive ventilation events with extubation after duplicates were removed")

There are 16243 invasive ventilation events with extubation after duplicates were removed


In [ ]:
# # Save as Parquet file for storage
# parquet_file_path = '/content/drive/MyDrive/MSc_Final_Project/mimic_data_extraction/mimic_extracted_data/unique_imv_extub_df_v01.parquet'

# final_df_unique_vent_extub.to_parquet(parquet_file_path, index=False)

**Section 2 - Number of patients in inclusion set and duration of ventilation**

From this set we can now determine the number of patients in our set and the duration of each ventilation event

In [ ]:
final_df_unique_vent_extub.head()

,subject_id,hadm_id,stay_id,ventilation_starttime,ventilation_endtime,ventilation_itemid,ventilation_ordercategoryname,extubation_starttime,extubation_endtime,extubation_itemid,extubation_ordercategoryname
0,10001884,26184834,37510196,2131-01-11 04:40:00,2131-01-12 17:40:00,225792,Ventilation,2131-01-12 17:40:00,2131-01-12 17:41:00,227194,Intubation/Extubation
1,10002428,23473524,35479615,2156-05-11 16:05:00,2156-05-20 10:45:00,225792,Ventilation,2156-05-20 10:45:00,2156-05-20 10:46:00,227194,Intubation/Extubation
2,10002428,28662225,38875437,2156-04-19 20:10:00,2156-04-22 17:05:00,225792,Ventilation,2156-04-22 17:10:00,2156-04-22 17:11:00,227194,Intubation/Extubation
3,10002760,28094813,31831386,2141-04-20 14:36:00,2141-04-21 08:00:00,225792,Ventilation,2141-04-21 08:00:00,2141-04-21 08:01:00,227194,Intubation/Extubation
4,10004235,24181354,34100191,2196-02-24 16:52:00,2196-02-27 16:28:00,225792,Ventilation,2196-02-27 16:28:00,2196-02-27 16:29:00,227194,Intubation/Extubation


In [ ]:
# Calculate the ventilation duration in minutes
final_df_unique_vent_extub['ventilation_duration'] = (final_df_unique_vent_extub['ventilation_endtime'] - final_df_unique_vent_extub['ventilation_starttime']).dt.total_seconds() / 60

# Select the required columns for focused dataframe
ventilation_durations_df = final_df_unique_vent_extub[['subject_id', 'stay_id', 'ventilation_duration']]

ventilation_durations_df.head()

<ipython-input-55-47879e84360c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_unique_vent_extub['ventilation_duration'] = (final_df_unique_vent_extub['ventilation_endtime'] - final_df_unique_vent_extub['ventilation_starttime']).dt.total_seconds() / 60


,subject_id,stay_id,ventilation_duration
0,10001884,37510196,2220.0
1,10002428,35479615,12640.0
2,10002428,38875437,4135.0
3,10002760,31831386,1044.0
4,10004235,34100191,4296.0


In [ ]:
# Save final full file for later use
file_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/mimic_data_analysis/datasets/inclusion_set/final_inclusion_set_full.parquet'

final_df_unique_vent_extub.to_parquet(file_path, index=False)

Determine how many stays each patient has had under IMV

In [ ]:
final_df_unique_vent_extub['ventilation_duration'] = (final_df_unique_vent_extub['ventilation_endtime'] - final_df_unique_vent_extub['ventilation_starttime']).dt.total_seconds() / 60

ventilation_durations_df = final_df_unique_vent_extub[['subject_id', 'stay_id', 'ventilation_starttime', 'ventilation_duration']]

# Sort the dataframe by subject_id and ventilation_starttime
ventilation_durations_df = ventilation_durations_df.sort_values(by=['subject_id', 'ventilation_starttime'])

# Column to highlight the order of each stay for each patient
ventilation_durations_df['stay_number'] = ventilation_durations_df.groupby('subject_id').cumcount() + 1

<ipython-input-56-d48567e71898>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_unique_vent_extub['ventilation_duration'] = (final_df_unique_vent_extub['ventilation_endtime'] - final_df_unique_vent_extub['ventilation_starttime']).dt.total_seconds() / 60


In [ ]:
ventilation_durations_df.head()

,subject_id,stay_id,ventilation_starttime,ventilation_duration,stay_number
0,10001884,37510196,2131-01-11 04:40:00,2220.0,1
2,10002428,38875437,2156-04-19 20:10:00,4135.0,1
1,10002428,35479615,2156-05-11 16:05:00,12640.0,2
3,10002760,31831386,2141-04-20 14:36:00,1044.0,1
4,10004235,34100191,2196-02-24 16:52:00,4296.0,1


In [ ]:
# Remove starttime column
ventilation_durations_df = ventilation_durations_df.drop(columns=['ventilation_starttime'])

ventilation_durations_df.head()

,subject_id,stay_id,ventilation_duration,stay_number
0,10001884,37510196,2220.0,1
2,10002428,38875437,4135.0,1
1,10002428,35479615,12640.0,2
3,10002760,31831386,1044.0,1
4,10004235,34100191,4296.0,1


In [ ]:
# Highlight duration is calculated in minutes
ventilation_durations_df = ventilation_durations_df.rename(columns={'ventilation_duration': 'ventilation_duration_minutes'})

ventilation_durations_df.head()

,subject_id,stay_id,ventilation_duration_minutes,stay_number
0,10001884,37510196,2220.0,1
2,10002428,38875437,4135.0,1
1,10002428,35479615,12640.0,2
3,10002760,31831386,1044.0,1
4,10004235,34100191,4296.0,1


In [ ]:
# Check no data has been lost
ventilation_durations_df.shape[0]

16243

Count the number of patients in the inclusion set

In [ ]:
no_patients = ventilation_durations_df['subject_id'].nunique()

print(f"Number of unique subject IDs: {no_patients}")

Number of unique subject IDs: 14315


Save ventilation duration dataframe for later use

In [ ]:
# parquet_file_path = '/content/drive/MyDrive/MSc_Final_Project/mimic_data_extraction/mimic_extracted_data/vent_durations_df_v01.parquet'

# ventilation_durations_df.to_parquet(parquet_file_path, index=False)

In [ ]:
# # Load parquet file as CSV for visualisation
# parquet_file_path = '/content/drive/MyDrive/MSc_Final_Project/mimic_data_extraction/mimic_extracted_data/vent_durations_df_v01.parquet'

# temp_df = pd.read_parquet(parquet_file_path)

# csv_file_path = '/content/drive/MyDrive/MSc_Final_Project/mimic_data_extraction/mimic_extracted_data/vent_durations_df_v01.csv'

# temp_df.to_csv(csv_file_path, index=False)